In [ ]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import random
import os
import time
import re

# --- 画像処理を行う関数 (変更なし) ---
def split_and_rearrange_flag(image_url, grid_size=6):
    """
    指定されたURLから画像をダウンロードし、タイルに分割後、
    ランダムに並べ替えた新しい画像オブジェクトを返します。
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    }
    try:
        response = requests.get(image_url, headers=headers)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert('RGB')
        
        width, height = img.size
        tile_width = width // grid_size
        tile_height = height // grid_size
        
        tiles = []
        for y in range(grid_size):
            for x in range(grid_size):
                box = (x * tile_width, y * tile_height, (x + 1) * tile_width, (y + 1) * tile_height)
                tile = img.crop(box)
                tiles.append(tile)
        
        #random.shuffle(tiles)
        
        new_image = Image.new('RGB', (width, height))
        
        tile_index = 0
        for y in range(grid_size):
            for x in range(grid_size):
                paste_position = (x * tile_width, y * tile_height)
                new_image.paste(tiles[tile_index], paste_position)
                tile_index += 1
                
        return new_image
    except Exception as e:
        # エラーが発生した場合はNoneを返す
        print(f"  └── ERROR: {e}")
        return None

def sanitize_filename(name):
    """ファイル名として使えない文字をアンダースコアに置換する"""
    # Windows/Mac/Linuxで共通して使えない文字を除去
    return re.sub(r'[\\/*?:"<>|]', "_", name)

# --- ★★★ ここからがメインの実行部分 ★★★ ---
if __name__ == "__main__":
    # 1. 設定
    csv_file = 'flags_database_success.csv'  # 読み込むCSVファイル名
    output_dir = 'flags_output'   # 画像を保存するフォルダ名

    # 2. 保存用フォルダを作成
    os.makedirs(output_dir, exist_ok=True)
    print(f"画像の保存先フォルダ: '{output_dir}'")

    # 3. CSVファイルを読み込む
    try:
        df = pd.read_csv(csv_file)
        print(f"'{csv_file}'から {len(df)} 件の国旗データを読み込みました。")
    except FileNotFoundError:
        print(f"エラー: CSVファイル '{csv_file}' が見つかりません。スクリプトと同じ場所に配置してください。")
        exit()

    print("-" * 30)
    
    # 4. CSVの各行をループして、1件ずつ画像処理を実行
    for index, row in df.iterrows():
        country_name = row['国名']
        svg_url = row['SVG画像URL']
        
        print(f"[{index + 1}/{len(df)}] {country_name} の処理を開始...")
        
        # PNG URLを生成
        png_url = svg_url.replace('/commons/', '/commons/thumb/') + '/600px-' + os.path.basename(svg_url) + '.png'
        
        # ★★★ ここで関数を呼び出し ★★★
        rearranged_img = split_and_rearrange_flag(png_url)
        
        # 5. 結果をファイルとして保存
        if rearranged_img:
            # 国名からファイル名として安全な文字列を生成
            safe_filename = sanitize_filename(country_name)
            save_path = os.path.join(output_dir, f"{safe_filename}_rearranged.png")
            rearranged_img.save(save_path)
            print(f"  └── SUCCESS: '{save_path}' として画像を保存しました。")
        else:
            print(f"  └── FAILED: {country_name} の画像処理に失敗しました。")
        
        # サーバーに負荷をかけないための待機
        time.sleep(0.5)

    print("-" * 30)
    print("全ての処理が完了しました。")